### Read data and preprocessing

In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('annotations.xml')
root = tree.getroot()

In [2]:
root.tag

'annotations'

In [3]:
helmet = {"yes":0, "no": 1}
mask = {"yes":0, "no": 1, "invisible":2, "wrong":3}

def switch_case(helmet_status, mask_status):
    if helmet_status == 0 and mask_status == 0:
        return 0
    elif helmet_status == 0 and mask_status == 1:
        return 1
    elif helmet_status == 0 and mask_status == 2:
        return 2
    elif helmet_status == 0 and mask_status == 3:
        return 3
    elif helmet_status == 1 and mask_status == 0:
        return 4
    elif helmet_status == 1 and mask_status == 1:
        return 5
    elif helmet_status == 1 and mask_status == 2:
        return 6
    elif helmet_status == 1 and mask_status == 3:
        return 7

In [4]:
import cv2

In [5]:
X = []
Y = []
for child in root.iter('image'):
    name = child.get('id') + ".jpg"
    img = cv2.imread("images/"+ name)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for box_child in child.findall('box'):
        class_name = box_child.get("label")
        if class_name == "head":
            x1 = int(float(box_child.get("xtl")))
            y1 = int(float(box_child.get("ytl")))
            x2 = int(float(box_child.get("xbr")))
            y2 = int(float(box_child.get("ybr")))
            if (y2-y1) > 16 and (x2 -x1) > 16:
                X.append(cv2.resize(rgb_img[y1:y2,x1:x2], (64, 64))/255.0)
                label = [0] * len(helmet) * len(mask)
                helmet_status = 0
                mask_status = 0
                for labels in box_child.iter("attribute"):
                    if labels.get("name") == "has_safety_helmet":
                        helmet_status = helmet[labels.text]
                    elif labels.get("name") == "mask":
                        mask_status = mask[labels.text]
                label[switch_case(helmet_status, mask_status)] = 1
                Y.append(label)

In [6]:
len(X)

995

In [7]:
X[0].shape

(64, 64, 3)

In [8]:
from matplotlib import pyplot as plt
plt.imshow(X[0])

In [9]:
Y[0]

[1, 0, 0, 0, 0, 0, 0, 0]

In [10]:
import numpy as np
np.array(X).shape

(995, 64, 64, 3)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [13]:
print(np.array(X_train).shape)
print(np.array(Y_train).shape)
print(np.array(X_val).shape)
print(np.array(Y_val).shape)
print(np.array(X_test).shape)
print(np.array(Y_test).shape)

(636, 64, 64, 3)
(636, 8)
(160, 64, 64, 3)
(160, 8)
(199, 64, 64, 3)
(199, 8)


### Build a classification model

In [14]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras import optimizers

Using TensorFlow backend.
/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tapan/an

In [31]:
model = Sequential()
model.add(Convolution2D(16, 3, 3, input_shape=(64,64,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(8,activation = "softmax"))

/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(64, 64, 3...)`
  
/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/home/tapan/anaconda3/envs/mask_rcnn/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [32]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.01),metrics=['accuracy'])

### Model training

In [33]:
model.fit(np.array(X_train),np.array(Y_train), batch_size = 32, epochs=20, verbose=1, validation_data=(np.array(X_val),np.array(Y_val)), shuffle = True)

Train on 636 samples, validate on 160 samples
Epoch 1/20
636/636 [==============================] - 1s 2ms/step - loss: 2.4007 - accuracy: 0.2374 - val_loss: 1.6189 - val_accuracy: 0.3438
Epoch 2/20
636/636 [==============================] - 1s 2ms/step - loss: 1.7123 - accuracy: 0.2814 - val_loss: 1.6199 - val_accuracy: 0.3313
Epoch 3/20
636/636 [==============================] - 1s 1ms/step - loss: 1.6976 - accuracy: 0.3365 - val_loss: 1.5206 - val_accuracy: 0.4000
Epoch 4/20
636/636 [==============================] - 1s 2ms/step - loss: 1.5643 - accuracy: 0.3836 - val_loss: 1.4745 - val_accuracy: 0.4187
Epoch 5/20
636/636 [==============================] - 1s 1ms/step - loss: 1.4808 - accuracy: 0.4167 - val_loss: 1.4230 - val_accuracy: 0.4688
Epoch 6/20
636/636 [==============================] - 1s 1ms/step - loss: 1.4133 - accuracy: 0.4198 - val_loss: 1.4696 - val_accuracy: 0.4125
Epoch 7/20
636/636 [==============================] - 1s 1ms/step - loss: 1.3720 - accuracy: 0.4638 - 

### Evaluation

In [34]:
model.evaluate(np.array(X_test), np.array(Y_test))

199/199 [==============================] - 0s 509us/step


[2.051135962931954, 0.4271356761455536]

In [35]:
y_pred = model.predict(np.array(X_test))

In [36]:
y_prediction = np.argmax(y_pred, axis=1)

In [37]:
y_actual = np.argmax(np.array(Y_test), axis=1)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(y_actual, y_prediction)

array([[18, 10,  4, 13,  2,  2],
       [ 7, 34, 10,  8,  0,  2],
       [ 1, 12, 13,  1,  0,  3],
       [ 3,  3,  0, 15,  0,  0],
       [ 1,  4,  5,  4,  0,  6],
       [ 4,  3,  0,  3,  3,  5]])

### justification for evaluation matric

As per the data disribution, most of the classes are about wearing **mask status (yes , no)**, **sometimes invisible** and **rarely wrong**. For helmet status (yes , no) is equally distribute.

After merging these two status, 8 classes can be extracted by encoding two status.

So, due to data imbalance confusion matrix has been used.